In [1]:
import os
import numpy as np
import pandas as pd
import xarray as xr

from pointprocess import *
import pointprocess.plotting as pplot
from lightning_setup import *
%matplotlib inline
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

In [2]:
c = Region(city=cities['cedar'])
c.SUBSETTED = False
c.CENTER = (37.7, -111.8)
c.RADIUS = 0.7
c.define_grid(step=1, units='km')

xx, yy = np.meshgrid(c.gridx[1:], c.gridy[1:])
xx = xx.flatten()
yy = yy.flatten()
X = np.stack([yy, xx])

In [3]:
storm = '2011-08-21'

In [4]:
from titan import *
from likelihood import Equation_18, Equation_17, Equation_16, get_l
fname = './input/{storm}.txt'.format(storm=storm)
titan_df = read_TITAN(fname)

In [5]:
tr = titan_df.index.unique().sort_values()
tr = tr.insert(0, tr[0]-pd.Timedelta(minutes=3))

In [6]:
ds = c.get_daily_ds(storm, filter_CG=dict(method='CG'))

In [7]:
# slice according to the titan times
box, tr = c.get_grid_slices(ds, tr=tr)
tr = tr[1:]

In [8]:
print('The shape of the data is in the order t, x, y', box.shape)

The shape of the data is in the order t, x, y (102, 100, 124)


In [9]:
%%time
Equation_18(box)

CPU times: user 10.8 ms, sys: 9.97 ms, total: 20.8 ms
Wall time: 19.5 ms


801.22501282631652

In [10]:
mini_df = titan_df[['EnvelopeArea(km2)', 'ReflCentroidLat(deg)', 'ReflCentroidLon(deg)', 'MaxDBZ(dBZ)', 'SimpleNum']]
mini_df.columns = ['Ai', 'lat', 'lon', 'Zi', 'i']

In [11]:
%%time
s = get_l((mini_df['lat'], mini_df['lon']), c.gridy, c.gridx)
mini_df = mini_df.assign(**dict(list(zip(['yloc', 'xloc', 'l'], s))))

CPU times: user 7.38 ms, sys: 381 µs, total: 7.76 ms
Wall time: 5.85 ms


In [12]:
strikes_per_dbz_45 = box.sum()/(mini_df.Ai*(mini_df.Zi-45)).sum()
strikes_per_dbz_45

0.010726273253822506

In [14]:
%%time
import scipy.optimize as optimize

def f(theta):
    gamma_kl = Equation_16(c.gridy, c.gridx, tr, mini_df, gamma=theta[0], alpha=theta[1], beta=theta[2])
    return Equation_17(box, gamma_kl)
result = optimize.minimize(f, [1, strikes_per_dbz_45, 45], tol=10)
print(result)

NameError: name 'box' is not defined

### Unused

In [ ]:
plt.figure(figsize=(10,8))
c.plot_grid(gamma_kl.values.reshape(box.shape)[50], cmap=cmap, cbar=True, vmin=.1)
c.plot_grid(box[50], cmap=cmap, cbar=False, vmin=.1);

In [ ]:
def at_t(t, dfi, gamma=1, alpha=0, beta=1):
    # Benchmarking: 839 µs
    Yi = (dfi.loc[t,'lat'], dfi.loc[t, 'lon'])
    Ai = dfi.loc[t, 'Ai']
    Zi = dfi.loc[t, 'Zi']
    xloc = dfi.loc[t, 'x']
    yloc = dfi.loc[t, 'y']

    sigmai = Equation_12(Ai, gamma)
    Ii = Equation_13(Zi-45, alpha, beta)
    
    # Benchmarking: 103 µs
    xx, yy = np.meshgrid(c.gridx[max(0, int(xloc-Ai)): min(int(xloc+Ai), c.gridx.shape[0])],
                         c.gridy[max(0, int(yloc-Ai)): min(int(yloc+Ai), c.gridy.shape[0])])
    X = np.stack([yy.flatten(), xx.flatten()])

    # Benchmarking: 3.7 ms
    g = np.apply_along_axis(Equation_11, 0, X, Yi=Yi, sigmai=sigmai, Ii=Ii)
    
    #TODO but then we have a mis-shapen g_t
    return g

In [ ]:
def at_t(t, dfi, gamma=1, alpha=0, beta=1):
    # Benchmarking: 839 µs per t
    Yi = (dfi.loc[t,'lat'], dfi.loc[t, 'lon'])
    sigmai = Equation_12(dfi.loc[t, 'Ai'], gamma)
    Ii = Equation_13(dfi.loc[t, 'Zi']-45, alpha, beta)

    # Benchmarking: 465 ms per t
    g = np.apply_along_axis(Equation_11, 0, X, Yi=Yi, sigmai=sigmai, Ii=Ii)
    return g

In [ ]:
%%time
get_l((mini_df.lat[0], mini_df.lon[0]), c.gridy, c.gridx)

In [ ]:
mini_df.lat[0], mini_df.lon[0]

In [ ]:
box.shape[2]

In [ ]:
c.gridx.shape

In [ ]:
%%time
# if lat, lon are within bounds, get the grid cell they fall in.
def get_loc(ll, grid_ll):
    if x<grid_ll[-1] and x>grid_ll[0]:
        return np.argmax(grid_ll>x)
    else:
        return None
    
mini_df = mini_df.assign(x=mini_df.lon.apply(func, grid_lon=c.gridx),
                         y=mini_df.lat.apply(func, grid_lon=c.gridy))

In [ ]:
mini_df = mini_df.assign(l=((mini_df.y-1)*(box.shape[2])+(mini_df.x)-1))
mini_df.head()

In [ ]:
def get_l(x):
    